In [4]:
import os
from dotenv import load_dotenv
from groq import Groq

In [5]:
#para rodar tudo, é necessário ter uma chave API do Groq, que pode ser obtida em https://groq.io/developers
#a minha está em um arquivo dotenv para que não seja exposta, mas você pode colocar a sua diretamente no código
load_dotenv()

groq_api_key = os.getenv("GROQ_API_KEY")

In [6]:
client = Groq(
    api_key=groq_api_key,
)

#note que o groq pega uma lista de mensagens, não uma única string
chat_completion = client.chat.completions.create(
    #primeiro parametro é a lista de mensagens
    messages=[

        {

            "role": "user",

            "content": "Explain the importance of fast language models",

        }

    ],
    #segundo parametro é o modelo de linguagem que será usado (é possível setar outro modelo que a API hospeda)
    model="llama-3.3-70b-versatile",
    
)


print(chat_completion.choices[0].message.content)


Fast language models are crucial in natural language processing (NLP) and have numerous applications in various fields. Here are some reasons why they are important:

1. **Improved Responsiveness**: Fast language models enable real-time or near-real-time responses, which is essential for applications like chatbots, virtual assistants, and conversational interfaces. Quick responses enhance user experience and increase engagement.
2. **Efficient Processing**: Fast language models can process large amounts of text data quickly, making them suitable for applications that require high-throughput, such as text classification, sentiment analysis, and language translation.
3. **Reduced Latency**: In applications like speech recognition, fast language models can reduce latency, enabling faster recognition and response to user input. This is particularly important in situations where timely response is critical, like in voice-controlled systems or real-time translation.
4. **Increased Scalabilit

In [7]:
#Agora vamos criar uma classe de Agente
class Agent:
    def __init__(self, client, system):
        self.client = client #o cliente do groq
        self.system = system #o system prompt
        self.messages = [] #lista de mensagens
        if self.messages is not None:
            self.messages.append({"role": "system", "content": self.system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content


In [8]:
system_prompt = """
You run in a loop of Thought, PAUSE, Action, PAUSE, Observation, PAUSE.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
Returns the mass of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of the Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with:

Observation: 5.972e24

Thought: Now I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

If you have the answer, output it as the Answer

Answer: The mass of Earth times 2 is 1.1944e25 kg

Now it's your turn!

""".strip()

In [9]:
#tools
def calculate(operation):
    return eval(operation)


def get_planet_mass(planet):
    match planet.lower():
        case "mercury":
            return 3.285e23
        case "venus":
            return 4.867e24
        case "earth":
            return 5.972e24
        case "mars":
            return 6.39e23
        case "jupiter":
            return 1.898e27
        case "saturn":
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "neptune":
            return 1.024e26
        case _:
            return 0.0



In [10]:
#inciando o agente
neil_tyson = Agent(client, system_prompt)

In [11]:
result = neil_tyson("What is the mass of Earth times 5?")

print(result)

Thought: To find the mass of Earth times 5, I first need to determine the mass of Earth. 
Action: get_planet_mass: Earth
PAUSE


In [12]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, PAUSE, Action, PAUSE, Observation, PAUSE.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns the mass of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of the Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with:\n\nObservation: 5.972e24\n\nThought: Now I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nIf you have the answer, output it as the Answer\n\nAnswer: The mas

In [13]:
#Rerodando o agente para ir para a ação
result = neil_tyson()
print(result)

In [14]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


In [15]:
#rodar dnv o agente com a observação 
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of Earth, I can calculate the mass of Earth times 5 by multiplying the observed mass by 5.
Action: calculate: 5.972e+24 * 5
PAUSE


In [16]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, PAUSE, Action, PAUSE, Observation, PAUSE.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nReturns the mass of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of the Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with:\n\nObservation: 5.972e24\n\nThought: Now I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nIf you have the answer, output it as the Answer\n\nAnswer: The mas

In [17]:
result = neil_tyson()
print(result)

In [18]:
#loop
import regex as re

def agent_loop(max_iterations, system_prompt, query):
    agent = Agent(client, system_prompt)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query #a primeira mensagem é a query
    i=0
    while i < max_iterations:
        i+=1
        result = agent(next_prompt) #toda vez essa função vai retornar a próxima mensagem
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE) #vai retornar o nome da ação e os parametros
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result}"

            else:
                next_prompt = f"Observation: Tool not found"

            print(next_prompt)
            continue
        if "Answer" in result:
            break


In [19]:
agent_loop(10, system_prompt, "What is the mass of Earth plus the mass of Mercury, times 5?")

Thought: To solve this problem, I need to find the mass of Earth and the mass of Mercury, add them together, and then multiply the result by 5. First, I'll find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: Now that I have the masses of both Earth and Mercury, I can add them together and then multiply the result by 5. First, I'll add the masses of Earth and Mercury.

Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now that I have the sum of the masses of Earth and Mercury, I can multiply this result by 5 to get the final answer.

Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have now calculated the mass of Earth plus the mass of Mercury, times 5, so I can provide the final answer.

Answer: The mass of Earth pl

# Ciclo de um ReAct Agent

Primeiramente, nosso agente irá analisar o prompt do sistema, que deve ser algo do tipo:
    "Você trabalhará sempre no seguinte ciclo: primeiro você pensa sobre o problema dado, então você faz a ação necessária, pausa e e verifica se está condizente com o problema. Fique nesse loop até encontrar a resposta final, na qual você entregará ao sistema"

Uma coisa importante para se adicionar ao prompt inicial são as ferramentas disponíveis para uso, para que o sistema fique completo.

![A imagen se trata de um print recortado do vídeo, pois o dono do mesmo não colocou o fluxo na descrição e achei muito interessante para visualização](ReACtDiagSimp.png "Diagrama simplificado")

![Aqui temos a versão detalhada desse fluxo](ReACtDiag.png "Diagrama detalhado do passo a passo com apenas uma ferramenta ligada ao agente")

Basicamente, temos:
1- Usuário pede algo
2- Agente utiliza LLM para pensar sobre o problema que precisa resolver
3- Utiliza as ferramentas disponíveis para poder solucionar o problema da maneira mais precisa possível
4- Coleta a resposta da ferramenta, verifica se a resposta é condizente com o pedido que foi feito
5- Envia a resposta ao usuário se estiver satisfatório, caso contrário refaz os passos 2, 3 e 4.

# Início da aplicação prática.

## O primeiro passo é conseguir uma chave API no groq.

Temos que entrar no link do site do Groq
[Groq API KEY](https://console.groq.com/playground)

## Agora, seguimos para a criação do agente

Na documentação do groq, tem uma aba de quickstart. Copiaremos o código de la para iniciar a criação do agente.

Primeiro nós testamos a api para ver se está funcionando. Estando tudo ok, partiremos para a criação do Agente. Primeiro criamos a classe agente, e definimos algumas funções do agente.

### Class Agent()
Temos o agente, e quando iniciamos temos um client se inicializando e o system prompt (loop). Então criamos a lista de mensagens e se essa lista não estiver vazia, nós vamos incluí-la na lista como uma mensagem do sistema, em que terão as instruções primordiais do seu agente.

Então temos o metodo call que vai ser acionado toda vez que instanciarmos o Agent (classe). Temos duas possibilidades pra call do agente. A primeira é se caso chamarmos o agente com uma mensagem, então irá armazenar (.append) a mensagem na nossa lista 'messages=[]' e iremos executar a lm, que basicamente irá rodar todas as mensagens no modelo de linguagem (função execute) e irá armazenar tudo no resultado da call do modelo de linguagem e vai retornar esse resultado

### System Prompt
É um loop que deve fazer o LLM pensar, fazer a ação necessária, pausar, observar se aquilo que fez está correto e só parar esse loop se estiver. O prompt feito é baseado completamente na ideia do ReAcT.

### Tools
É basicamente as funções que devemos adicionar dos prompts feitos. Creio que se tivéssemos utilizando algum framework para isso, seria muito mais otimizado esse processo de fazer o setup das ferramentas

### Rodando o agente manualmente
Para rodar o agente manualmente, fizemos um passo a passo geral do nosso agente, manualmente.

### Criando um loop para rodar automaticamente
Para não ficar dando recall no agente toda vez, nós faremos o processo virar um loop para o agente pensar gradualmente até achar a solução por conta própria.

Para funcionar, devemos chamar o agente em um número máximo de iterações, no vídeo utilizamos 10 iterações como exemplo, e iremos rodar o agente e analisar o seu retorno todas as vezes.

Basicamente, estamos inicializando o agente, depois damos o nome das nossas ferramentas e daí iniciamos o nosso loop que irá chamar o agente e checar o resultado (passo a passo). Se o resultado incluir PAUSE e Action, deverá pegar o nome da ação (utilizando ReGEx), então o agente deve escolher a ferramenta a ser utilizada, e se conter nas ferramentas que o agente possui, ele executa, caso contrário, ele retorna que não possui a ferramenta ou que não foi possível selecionar a ferramenta correta.